In [1]:
import numpy as np
import skfuzzy as fuzz
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from scipy.optimize import differential_evolution, NonlinearConstraint, Bounds, minimize, fmin, rosen

In [55]:
# Generate universe variables
#sources: "https://github.com/scikit-fuzzy/scikit-fuzzy/blob/master/skfuzzy/membership/generatemf.py"

#   * Light on subjective ranges [0, 100]
#   * Motion on subjective ranges [0, 25] in units of percentage points
#   * Shutter Speed has a range of [0, 20] in small steps (due to the data)
x_light = np.arange(0, 101, 1)
x_motion = np.arange(0, 26, 1)
x_ss  = np.arange(0, 20, 0.0001)                                        


#well this function takes a vector/list of variables for mf boundies with input & spits out a number for sin()
def Sh(X, l, m):
    light_lo = fuzz.gaussmf(x_light,  X[0], X[1])
    light_md = fuzz.gaussmf(x_light,  X[2], X[3])
    light_hi = fuzz.gaussmf(x_light,  X[4], X[5])

    motion_st = fuzz.gaussmf(x_motion,  X[6], X[7])
    motion_mo = fuzz.gaussmf(x_motion,  X[8], X[9])
    motion_hi = fuzz.gaussmf(x_motion,  X[10], X[11])

    ss_lo = fuzz.gaussmf(x_ss,  X[12], X[13])
    ss_md = fuzz.gaussmf(x_ss,  X[14], X[15])
    ss_hi = fuzz.gaussmf(x_ss,  X[16], X[17])
    
    light_level_lo = fuzz.interp_membership(x_light, light_lo, l)
    light_level_md = fuzz.interp_membership(x_light, light_md, l)
    light_level_hi = fuzz.interp_membership(x_light, light_hi, l)

    motion_level_st = fuzz.interp_membership(x_motion, motion_st, m)
    motion_level_mo = fuzz.interp_membership(x_motion, motion_mo, m)
    motion_level_hi = fuzz.interp_membership(x_motion, motion_hi, m)
    
    
    active_rule1 = np.fmin(light_level_lo, motion_level_st)
    
    ss_activation_lo = np.fmax(np.fmin(active_rule1, ss_lo), 1-active_rule1)
    
    
    
    # For rule 2 & 3 :
    active_rule2 = np.fmin(light_level_md, motion_level_st)
    ss_activation_md2 = np.fmax(np.fmin(active_rule2, ss_md), 1-active_rule2)

    active_rule3 = np.fmin(light_level_hi, motion_level_st)
    ss_activation_hi3 = np.fmax(np.fmin(active_rule3, ss_hi), 1-active_rule3)

    active_rule4 = np.fmin(light_level_lo, motion_level_mo)
    ss_activation_md4 = np.fmax(np.fmin(active_rule4, ss_md), 1-active_rule4)

    active_rule5 = np.fmin(light_level_md, motion_level_mo)
    ss_activation_hi5 = np.fmax(np.fmin(active_rule5, ss_hi), 1-active_rule5)

    active_rule6 = np.fmin(light_level_hi, motion_level_hi)
    ss_activation_hi6 = np.fmax(np.fmin(active_rule5, ss_hi), 1-active_rule6)

    active_rule7 = np.fmin(light_level_lo, motion_level_hi)
    ss_activation_md7 = np.fmax(np.fmin(active_rule7, ss_md), 1-active_rule7)

    active_rule8 = np.fmin(light_level_md, motion_level_hi)
    ss_activation_hi8 = np.fmax(np.fmin(active_rule8, ss_hi), 1-active_rule8)

    active_rule9 = np.fmin(light_level_hi, motion_level_mo)
    ss_activation_md9 = np.fmax(np.fmin(active_rule9, ss_md), 1-active_rule9)

    ss_activation_md = np.fmax(np.fmax(ss_activation_md2, ss_activation_md4), np.fmax(ss_activation_md7, ss_activation_md9))

    ss_activation_hi = np.fmax(np.fmax(ss_activation_hi3, ss_activation_hi5), np.fmax(ss_activation_hi6, ss_activation_hi8))

    agg = np.fmin(np.fmin(ss_activation_lo, ss_activation_md), ss_activation_hi)
        
    ss = fuzz.defuzz(x_ss, agg, 'centroid')
    
    if agg.sum() == 0:
        return 10

    y = fuzz.defuzz(x_ss, agg, 'centroid')
    
    return y

In [56]:
Sh([7, 7, 50, 30, 3, 80, 30, 2, 3, 8, 7, 18, 10, 2, 3, 7, 7, 18, 10], 60, 16)

9.999950000011266

In [5]:
data = pd.read_excel (r'Final Project/Data_Camera2.xlsx')


In [6]:
data['Light'].iloc[7]

0

In [31]:
Li = []
Ob = []
Sh = []
for i in range(48):
    Li.append(data['Light'].iloc[i])
    #Ob.append(data['Object Motion'].iloc[i])
    Sh.append(data['Shutter Speed'].iloc[i])
Ob = [1, 4, 7,10, 13, 16, 19, 22, 1
,4
,7
,10
,13
,16
,19
,22
,1
,4
,7
,10
,13
,16
,19
,22
,1
,4
,7
,10
,13
,16
,19
,22
,1
,4
,7
,10
,13
,16
,19
,22
,1
,4
,7
,10
,13
,16
,19
,22]

In [32]:
Dy = np.hstack((Li, Ob, Sh))
Dy = np.reshape(Dy, (48, 3), order='F')
np.random.shuffle(Dy)
n,d = Dy.shape

In [33]:
trsize = int(0.7*n)
vasize = int(0.85*n)
Dt = Dy[0:trsize, :] 
Dv = Dy[trsize:vasize, :] 
Dte = Dy[vasize:n, :] 
print(Dt.shape)
print(Dv.shape)
print(Dte.shape)

(33, 3)
(7, 3)
(8, 3)


In [65]:
def Error1(X):
    v = np.array([])
    for i in Dt:
        #d = Sh(X, i[0], i[1])-i[2]
        #d = Sh(X, i[0], i[1])
        d = i[2]
        v = np.append(v, d)
    return np.linalg.norm(v, 2)

In [66]:
bounds = [(-10, 40), (0, 35),
          (-10, 100), (0, 100),
          (-10, 150), (0, 160),
          (-10, 40), (0, 20),
          (-10, 100), (0, 40),
          (-10, 40), (0, 70),
          (-10, 40), (0, 19),
          (-10, 70), (0, 30),
          (-10, 100), (0, 50)]

In [67]:
xinitial = [7, 7, 50, 30, 3, 80, 30, 2, 3, 8, 7, 18, 10, 2, 3, 7, 7, 18, 10]
result1 = differential_evolution(Error1, bounds, x0 = xinitial)
result1.x, result1.fun

ValueError: operands could not be broadcast together with shapes (19,) (18,) 

In [61]:
Error1([7, 7, 50, 30, 3, 80, 30, 2, 3, 8, 7, 18, 10, 2, 3, 7, 7, 18, 10])

57.445339237275846